In [2]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import itertools
import os
import pandas as pd
import sys
path_to_package = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(path_to_package)

In [3]:
df = pd.read_csv('../data/close.csv', index_col=0)
df = df.iloc[-50:, :3]

window = 4
corr = df.rolling(window).cov()

columns = ['{}/{}'.format(m, n) for (m, n) in itertools.product(df.columns, repeat=2)]

corr_flattened = pd.DataFrame(index=columns).transpose()

for i in range(df.shape[0]):

    data = corr[df.shape[1]*i : df.shape[1]*(i+1)].to_numpy().flatten()
    index = [corr.index[df.shape[1]*i][0]]

    temp = pd.DataFrame(data=data, columns=index, index=columns).transpose()
    corr_flattened = pd.concat([corr_flattened, temp])

corr_flattened.tail()

,MMM/MMM,MMM/ABT,MMM/ABBV,ABT/MMM,ABT/ABT,ABT/ABBV,ABBV/MMM,ABBV/ABT,ABBV/ABBV
2020-12-23,1.113287,0.587608,0.604265,0.587608,0.391294,0.272667,0.604265,0.272667,0.490197
2020-12-24,0.486291,0.143307,0.126876,0.143307,0.168092,-0.033241,0.126876,-0.033241,0.138157
2020-12-28,0.127666,0.107266,-0.033032,0.107266,0.181092,-0.062607,-0.033032,-0.062607,0.077490
2020-12-29,0.170292,0.003433,-0.174224,0.003433,0.192134,0.152435,-0.174224,0.152435,0.476490
2020-12-30,0.151692,-0.079643,-0.314301,-0.079643,0.087358,0.163868,-0.314301,0.163868,0.945797


In [39]:
def append_corr_matrix(df: pd.DataFrame,
                       window: int,
                       ) -> pd.DataFrame:
    """
        Computes the sliding correlation matrix of a multidimensional time series, \ 
        timewise flattens it and extracts just the upper triangular part (since it is symmetric), \
        then appends it to the initial time series.
    """

    columns = ['{}/{}'.format(m, n) for (m, n) in itertools.combinations_with_replacement(df.columns, r=2)]
    corr = df.rolling(window).cov()
    corr_flattened = pd.DataFrame(index=columns).transpose()

    for i in range(df.shape[0]):

        ind = np.triu_indices(df.shape[1])
        data = corr[df.shape[1]*i : df.shape[1]*(i+1)].to_numpy()[ind]
        index = [corr.index[df.shape[1]*i][0]]

        temp = pd.DataFrame(data=data, columns=index, index=columns).transpose()
        corr_flattened = pd.concat([corr_flattened, temp])

    return pd.concat([df, corr_flattened], axis=1).iloc[window-1 : ]

In [41]:
corr_flattened = append_corr_matrix(df, 4)
corr_flattened.head()

,MMM,ABT,ABBV,MMM/MMM,MMM/ABT,MMM/ABBV,ABT/ABT,ABT/ABBV,ABBV/ABBV
2020-10-23,169.800003,107.790001,84.339996,0.678760,1.074071,0.447993,2.200203,0.917333,0.498424
2020-10-26,166.160004,109.000000,83.919998,3.884069,-0.273692,0.273001,2.503136,0.877366,0.459098
2020-10-27,161.029999,108.580002,82.709999,19.309392,0.271319,3.272198,0.513426,0.024067,0.584198
2020-10-28,158.529999,106.419998,80.489998,25.663291,2.612271,8.095133,1.288628,1.469286,2.984432
2020-10-29,158.479996,105.000000,80.669998,13.022622,5.596673,5.673438,3.527603,2.850035,2.742826


In [47]:
a = corr_flattened.iloc[4]

In [51]:
a.shape[0]

9